In [1]:
from xml.etree.ElementInclude import include
# allows graphs to display
%matplotlib inline
# allows multiple outputs per cell
%config InteractiveShell.ast_node_interactivity = 'all'

# Load libraries and data
import duckdb
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# from sklearn.preprocessing import

DUCKDB_PATH = "/Users/bradybates/Desktop/personal_projects/LOCAL_BASEBALL_COMPUTER_DATASET/scripts/local.db"

In [18]:
class Query:
    query: str
    field_exclusions: list[str]
    field_inclusions: list[str]
    field_type_exclusions: list[str]
    field_type_inclusions: list[str]

    def __init__(
            self, query: str,
            field_exclusions: list[str] = None,
            field_inclusions: list[str] = None,
            field_type_exclusions: list[str] = None,
            field_type_inclusions: list[str] = None
        ):
        self.query = query
        self.field_exclusions = field_exclusions
        self.field_inclusions = field_inclusions
        self.field_type_exclusions = field_type_exclusions
        self.field_type_inclusions = field_type_inclusions

    def get_query(self):
        return self.query

    def get_field_type_exclusions(self):
        return self.field_type_exclusions

    # def __str__(self):
    #     return self.query
    #
    # def __repr__(self):
    #     return self.query

game_results = Query("""
    SELECT
      *,
      CASE WHEN home_team_id = winning_team_id THEN 1 ELSE 0 END AS home_winning_team,
      CASE WHEN away_team_id = winning_team_id THEN 1 ELSE 0 END AS away_winning_team
    FROM main_models.game_results
    ;
    """,
    field_type_exclusions=["boolean", "string"],
    field_type_inclusions=["number"]
)


In [3]:
# Check missing values
with duckdb.connect(DUCKDB_PATH) as connection:
  result = connection.execute("""
  SELECT *
  FROM main_models.game_results
  ;
  """).fetch_df()

result.info()
print(result.duplicated().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233439 entries, 0 to 233438
Data columns (total 28 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   game_id                     233439 non-null  object        
 1   season                      233439 non-null  int16         
 2   game_type                   233439 non-null  object        
 3   game_finish_date            233439 non-null  datetime64[us]
 4   home_team_id                233439 non-null  object        
 5   away_team_id                233439 non-null  object        
 6   winning_team_id             232192 non-null  object        
 7   losing_team_id              232192 non-null  object        
 8   winning_team_score          232192 non-null  float64       
 9   losing_team_score           232192 non-null  float64       
 10  winning_side                232192 non-null  object        
 11  losing_side                 232192 non-

In [4]:
### QUERY SCRATCHPAD ###
# result = connection.execute("""
# """).fetch_df().describe()
with duckdb.connect(DUCKDB_PATH) as connection:
    result = connection.execute(game_results.get_query()).fetch_df()

result.info()

# result2 = connection.execute("""
# SELECT
#   *,
#   CASE WHEN home_team_id = winning_team_id THEN 1 ELSE 0 END AS home_team_win,
#   CASE WHEN away_team_id = winning_team_id THEN 1 ELSE 0 END AS away_team_win,
#
# FROM main_models.game_results
# ;
# """).fetch_df().describe()


try:
    print(result)
    print("\n\nRESULT 2")
    print(result2)
except Exception as e:
    pass

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233439 entries, 0 to 233438
Data columns (total 30 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   game_id                     233439 non-null  object        
 1   season                      233439 non-null  int16         
 2   game_type                   233439 non-null  object        
 3   game_finish_date            233439 non-null  datetime64[us]
 4   home_team_id                233439 non-null  object        
 5   away_team_id                233439 non-null  object        
 6   winning_team_id             232192 non-null  object        
 7   losing_team_id              232192 non-null  object        
 8   winning_team_score          232192 non-null  float64       
 9   losing_team_score           232192 non-null  float64       
 10  winning_side                232192 non-null  object        
 11  losing_side                 232192 non-

In [21]:
with duckdb.connect(DUCKDB_PATH) as connection:
    df = connection.execute("""
    SELECT
      *,
      CASE WHEN home_team_id = winning_team_id THEN 1 ELSE 0 END AS home_winning_team
    FROM main_models.game_results
    ;
    """).fetch_df()

include = df.select_dtypes(game_results.field_type_inclusions, exclude=game_results.field_type_exclusions)

sns.heatmap(df.corr(), annot=True)
plt.show()

ValueError: could not convert string to float: 'WS1195304210'